In [1]:
from mec_env_var import *
from helper import *
import tensorflow as tf
import ipdb as pdb
import time

for i in range(10):
    
    print('---------' + str(i) + '------------')

    MAX_EPISODE = 1000
    MAX_EPISODE_LEN = 200

    NUM_T = 1
    NUM_R = 4
    SIGMA2 = 1e-9

    t_factor = 1.0
    noise_sigma = 0.12

    config = {'state_dim':3, 'action_dim':2};
    train_config = {'minibatch_size':64, 'actor_lr':0.0001, 'tau':0.001, 
                    'critic_lr':0.001, 'gamma':0.99, 'buffer_size':250000, 
                    'random_seed':int(time.clock()*1000%1000), 'noise_sigma':noise_sigma}
    user_config = [{'id':'1', 'model':'AR', 'num_r':NUM_R, 'rate':3.0, 'dis':100, 'action_bound':1, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000}]

    print(user_config)
    # 0. initialize the session object
    sess = tf.Session() 

    # gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

    # 1. include all user in the system according to the user_config
    user_list = [];
    for info in user_config:
        info.update(config)
        user_list.append(MecTermRL(sess, info, train_config))
        print('Initialization OK!----> user ' + info['id'])

    # 2. create the simulation env
    env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)

    sess.run(tf.global_variables_initializer())

    env.init_target_network()

    # #Create a saver object which will save all the variables
    # saver = tf.train.Saver() 

    res_r = []
    res_p = []
    res_b = []
    res_o = []
    # 3. start to explore for each episode
    for i in range(MAX_EPISODE):

        cur_init_ds_ep = env.reset()

        cur_r_ep = np.zeros(len(user_list))
        cur_p_ep = np.zeros(len(user_list))
        cur_ts_ep = np.zeros(len(user_list))
        cur_ps_ep = np.zeros(len(user_list))
        cur_rs_ep = np.zeros(len(user_list))
        cur_ds_ep = np.zeros(len(user_list))
        cur_ch_ep = np.zeros(len(user_list))
        cur_of_ep = np.zeros(len(user_list))

        for j in range(MAX_EPISODE_LEN):

            # first try to transmit from current state
            [cur_r, done, cur_p, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()

            cur_r_ep += cur_r
            cur_p_ep += cur_p
            cur_ts_ep += cur_ts
            cur_ps_ep += cur_ps
            cur_rs_ep += cur_rs
            cur_ds_ep += cur_ds
            cur_ch_ep += cur_ch
            cur_of_ep += cur_of


            if done:
                res_r.append(cur_r_ep/MAX_EPISODE_LEN)
                res_p.append(cur_p_ep/MAX_EPISODE_LEN)
                res_b.append(cur_ds_ep/MAX_EPISODE_LEN)
                res_o.append(cur_of_ep/MAX_EPISODE_LEN)
                print('%d:r:%s,p:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep))

    name = 't_10/test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
    np.savez(name, res_r, res_p, res_b, res_o)

    sess.close()

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
---------0------------
[{'id': '1', 'model': 'AR', 'num_r': 4, 'rate': 3.0, 'dis': 100, 'action_bound': 1, 'data_buf_size': 100, 't_factor': 1.0, 'penalty': 1000}]
Initialization OK!----> user 1
0:r:[-9.94222544],p:[0.99422254],tr:[1.52168755],pr:[1.55960041],rev:[2.88035507],dbuf:[10.57783407],ch:[12.92140528],ibuf:[2.0]
1:r:[-677.25410303],p:[0.2254103],tr:[0.4222908],pr:[0.52987532],rev:[3.35505862],dbuf:[88.83971118],ch:[10.84467162],ibuf:[36.0]
2:r:[-171.42559557],p:[0.64255956],tr:[0.86939735],pr:[1.28061874],rev:[2.75214448],dbuf:[80.70094975],ch:[12.51011711],ibuf:[12.5]
3:r:[-9.31372728],p:[0.93137273],tr:[1.21050449],pr:[1.56038145],rev:[2.85581439],dbuf:[25.30710313],ch:[10.34393972],ibuf:[21.0]
4:r:[-13.96835613],p:[1.39683561],tr:[1.66540846],pr:[1.67354015],rev:[3.09276523],dbuf:[21.24171414],ch:[9.60012765],ibuf:[43.5]
5:r:[-12.35042273],p:[1.23504227],tr:[1.35991041],pr:[

61:r:[-8.68620196],p:[0.8686202],tr:[1.42896512],pr:[1.25542952],rev:[2.79079548],dbuf:[51.91834396],ch:[13.03332746],ibuf:[23.0]
62:r:[-8.36175267],p:[0.83617527],tr:[1.62555984],pr:[1.19434849],rev:[2.88252186],dbuf:[50.11667581],ch:[13.90610661],ibuf:[30.0]
63:r:[-10.65474162],p:[1.06547416],tr:[1.08460919],pr:[1.64655388],rev:[3.02443013],dbuf:[49.45861288],ch:[10.79282655],ibuf:[16.5]
64:r:[-9.31738388],p:[0.93173839],tr:[1.61923914],pr:[1.30869019],rev:[3.06068267],dbuf:[49.2335642],ch:[12.11685027],ibuf:[37.0]
65:r:[-8.04389049],p:[0.80438905],tr:[1.70109241],pr:[0.98972008],rev:[2.88267609],dbuf:[51.48667176],ch:[16.50824342],ibuf:[24.0]
66:r:[-9.73180403],p:[0.9731804],tr:[1.73316581],pr:[1.30933011],rev:[3.01257791],dbuf:[55.90366412],ch:[16.5219188],ibuf:[42.5]
67:r:[-12.2153883],p:[1.22153883],tr:[1.69778191],pr:[1.48565225],rev:[3.29889237],dbuf:[55.9265265],ch:[11.44771961],ibuf:[22.5]
68:r:[-10.04830528],p:[1.00483053],tr:[1.26240749],pr:[1.55245444],rev:[2.98170776],dbu

124:r:[-9.86121697],p:[0.9861217],tr:[1.52241505],pr:[0.93442179],rev:[2.84173541],dbuf:[58.22778587],ch:[9.99511873],ibuf:[14.5]
125:r:[-11.3131581],p:[1.13131581],tr:[1.98655618],pr:[0.48247555],rev:[2.55615024],dbuf:[61.99936539],ch:[10.84612693],ibuf:[45.5]
126:r:[-10.41360608],p:[1.04136061],tr:[2.17923711],pr:[0.42279931],rev:[2.80158061],dbuf:[59.50347996],ch:[14.30733458],ibuf:[33.0]
127:r:[-11.2538121],p:[1.12538121],tr:[2.33318151],pr:[0.54230361],rev:[3.0092315],dbuf:[66.89669748],ch:[14.63543117],ibuf:[49.0]
128:r:[-13.68146843],p:[1.36814684],tr:[1.60632454],pr:[1.15492539],rev:[2.99761993],dbuf:[64.11989506],ch:[8.98347692],ibuf:[15.0]
129:r:[-9.84108494],p:[0.98410849],tr:[2.03472311],pr:[0.61501081],rev:[2.96246251],dbuf:[57.37229782],ch:[12.63091303],ibuf:[11.0]
130:r:[-11.76745877],p:[1.17674588],tr:[2.14573595],pr:[0.65758745],rev:[2.8997813],dbuf:[67.14298954],ch:[12.86125713],ibuf:[40.0]
131:r:[-12.22024161],p:[1.22202416],tr:[2.3115584],pr:[0.7762753],rev:[3.28553

187:r:[-8.61473909],p:[0.86147391],tr:[2.2614345],pr:[0.12109701],rev:[2.5216425],dbuf:[61.83051785],ch:[16.78313809],ibuf:[44.5]
188:r:[-10.99591638],p:[1.09959164],tr:[2.23147255],pr:[0.58052124],rev:[3.03673451],dbuf:[71.96670952],ch:[14.29840292],ibuf:[35.0]
189:r:[-19.55715996],p:[0.955716],tr:[1.61677546],pr:[1.01349993],rev:[2.95282013],dbuf:[71.83907272],ch:[12.59719499],ibuf:[13.5]
190:r:[-26.74776203],p:[1.1747762],tr:[1.60904101],pr:[1.49451754],rev:[3.25413713],dbuf:[79.69222285],ch:[12.0281228],ibuf:[43.5]
191:r:[-11.00895668],p:[1.10089567],tr:[1.24539586],pr:[1.55799968],rev:[3.07826232],dbuf:[71.69302441],ch:[10.75671835],ibuf:[26.0]
192:r:[-10.81344742],p:[1.08134474],tr:[1.613569],pr:[1.58501681],rev:[3.19168106],dbuf:[69.12652702],ch:[14.43715862],ibuf:[49.0]
193:r:[-8.19167877],p:[0.81916788],tr:[1.3414056],pr:[1.41050219],rev:[2.85939653],dbuf:[48.41828661],ch:[13.26837912],ibuf:[19.5]
194:r:[-9.79184143],p:[0.97918414],tr:[1.43494004],pr:[1.44966566],rev:[2.842224

250:r:[-9.50512072],p:[0.95051207],tr:[1.66490102],pr:[1.28362076],rev:[3.05848141],dbuf:[49.68933992],ch:[13.16900044],ibuf:[31.0]
251:r:[-7.28975329],p:[0.72897533],tr:[1.93429592],pr:[1.07294156],rev:[3.11678742],dbuf:[43.36225017],ch:[19.82353745],ibuf:[28.5]
252:r:[-9.45894933],p:[0.94589493],tr:[1.78031719],pr:[0.97321158],rev:[2.98800743],dbuf:[51.52547375],ch:[11.25801496],ibuf:[16.0]
253:r:[-8.81101161],p:[0.88110116],tr:[2.02376425],pr:[1.10629069],rev:[3.29314782],dbuf:[48.45909669],ch:[17.3611852],ibuf:[4.5]
254:r:[-8.35670974],p:[0.83567097],tr:[1.67221251],pr:[1.04343137],rev:[2.93176275],dbuf:[45.29718382],ch:[12.03556342],ibuf:[3.0]
255:r:[-9.6191362],p:[0.96191362],tr:[1.75136009],pr:[1.21364535],rev:[2.99698549],dbuf:[50.31711105],ch:[13.02703502],ibuf:[38.5]
256:r:[-7.77834864],p:[0.77783486],tr:[1.43276826],pr:[1.16469262],rev:[2.9458178],dbuf:[61.21942009],ch:[13.66993851],ibuf:[10.0]
257:r:[-7.48566417],p:[0.74856642],tr:[1.49235698],pr:[1.23333385],rev:[2.9344288

...

706:r:[-13.59285074],p:[1.35928507],tr:[1.43640865],pr:[1.96977192],rev:[3.38293379],dbuf:[39.35716155],ch:[15.47578108],ibuf:[49.0]
707:r:[-14.23826524],p:[1.42382652],tr:[1.65960935],pr:[1.92296215],rev:[3.07115599],dbuf:[6.38664076],ch:[14.77650826],ibuf:[1.5]
708:r:[-16.12767761],p:[1.61276776],tr:[1.78068274],pr:[1.99700349],rev:[2.98619202],dbuf:[7.08922546],ch:[14.03134666],ibuf:[27.5]
709:r:[-16.69834945],p:[1.66983494],tr:[1.95339605],pr:[1.96200241],rev:[3.67799987],dbuf:[16.81734205],ch:[13.85776327],ibuf:[13.5]
710:r:[-15.28328154],p:[1.52832815],tr:[2.05249777],pr:[1.89134917],rev:[3.2341466],dbuf:[10.33488225],ch:[16.94564473],ibuf:[43.0]
711:r:[-13.03867969],p:[1.30386797],tr:[1.43422603],pr:[1.90930978],rev:[2.6735031],dbuf:[13.36372724],ch:[14.0107654],ibuf:[36.0]
712:r:[-13.19831951],p:[1.31983195],tr:[1.71505855],pr:[1.84891997],rev:[3.43359093],dbuf:[19.83173232],ch:[15.59320586],ibuf:[28.0]
713:r:[-16.37683808],p:[1.63768381],tr:[1.90782917],pr:[1.9682586],rev:[3.3

770:r:[-20.],p:[2.],tr:[2.22609094],pr:[2.],rev:[2.63752743],dbuf:[6.62360144],ch:[12.91781908],ibuf:[38.0]
771:r:[-19.7046815],p:[1.97046815],tr:[2.46286782],pr:[1.99989713],rev:[3.37302721],dbuf:[7.44544166],ch:[17.01971835],ibuf:[12.0]
772:r:[-19.54833912],p:[1.95483391],tr:[2.16896664],pr:[2.],rev:[3.02007345],dbuf:[6.21698962],ch:[12.94390957],ibuf:[23.5]
773:r:[-19.55362673],p:[1.95536267],tr:[1.99388421],pr:[1.98339833],rev:[2.62388385],dbuf:[7.52005879],ch:[11.11245287],ibuf:[35.5]
774:r:[-19.56016084],p:[1.95601608],tr:[2.21736937],pr:[1.96979066],rev:[2.8865861],dbuf:[5.30017742],ch:[13.69797981],ibuf:[3.0]
775:r:[-20.],p:[2.],tr:[2.35946047],pr:[2.],rev:[2.81811788],dbuf:[7.80563982],ch:[14.88675617],ibuf:[45.0]
776:r:[-18.95943408],p:[1.89594341],tr:[1.87599195],pr:[2.],rev:[3.27364752],dbuf:[14.7003538],ch:[10.74388402],ibuf:[14.0]
777:r:[-19.54854873],p:[1.95485487],tr:[2.47125259],pr:[2.],rev:[3.28349486],dbuf:[8.39821181],ch:[17.89592635],ibuf:[22.5]
778:r:[-19.28557746

835:r:[-10.9439232],p:[1.09439232],tr:[1.16288575],pr:[1.77065989],rev:[2.56451621],dbuf:[8.86802847],ch:[16.42621916],ibuf:[40.5]
836:r:[-12.64837191],p:[1.26483719],tr:[1.1050625],pr:[1.82582217],rev:[2.86484158],dbuf:[12.1189455],ch:[13.80143312],ibuf:[13.5]
837:r:[-15.59270381],p:[1.55927038],tr:[1.48024435],pr:[1.91405161],rev:[3.32821483],dbuf:[17.42245283],ch:[11.92460973],ibuf:[47.5]
838:r:[-13.84323037],p:[1.38432304],tr:[1.20510271],pr:[1.82325271],rev:[2.88129506],dbuf:[15.41461663],ch:[9.96555642],ibuf:[42.0]
839:r:[-12.52222567],p:[1.25222257],tr:[1.16032589],pr:[1.8300906],rev:[2.94482092],dbuf:[9.15723568],ch:[11.51379237],ibuf:[0.5]
840:r:[-12.86936172],p:[1.28693617],tr:[1.15831647],pr:[1.85280735],rev:[2.78070417],dbuf:[12.22549326],ch:[14.67569678],ibuf:[37.5]
841:r:[-14.40636089],p:[1.44063609],tr:[1.32165043],pr:[1.86140684],rev:[2.89289861],dbuf:[12.62183969],ch:[10.74456802],ibuf:[42.0]
842:r:[-13.00088476],p:[1.30008848],tr:[0.9556033],pr:[1.95278536],rev:[2.720

898:r:[-12.6437105],p:[1.26437105],tr:[0.86411085],pr:[1.95709161],rev:[2.89326399],dbuf:[15.83489381],ch:[12.76166855],ibuf:[21.5]
899:r:[-14.19764199],p:[1.4197642],tr:[0.92687557],pr:[1.96984193],rev:[2.86494035],dbuf:[15.56128033],ch:[10.4566673],ibuf:[12.0]
900:r:[-13.95888527],p:[1.39588853],tr:[1.04517808],pr:[1.99417549],rev:[2.93698564],dbuf:[12.87227303],ch:[12.40603275],ibuf:[30.0]
901:r:[-13.91110987],p:[1.39111099],tr:[1.17884317],pr:[1.99998595],rev:[3.02150806],dbuf:[16.56173045],ch:[16.25382963],ibuf:[42.5]
902:r:[-14.16574787],p:[1.41657479],tr:[1.2267732],pr:[1.95569317],rev:[3.05613547],dbuf:[14.00552771],ch:[14.2129194],ibuf:[40.0]
903:r:[-14.06411675],p:[1.40641168],tr:[1.46419029],pr:[1.9654298],rev:[3.34114415],dbuf:[16.89324925],ch:[18.77128565],ibuf:[43.5]
904:r:[-13.57850376],p:[1.35785038],tr:[1.07228819],pr:[1.93552355],rev:[2.91931804],dbuf:[10.6931093],ch:[12.88042348],ibuf:[23.5]
905:r:[-14.27378508],p:[1.42737851],tr:[1.10793636],pr:[1.93603271],rev:[3.1

961:r:[-10.82392432],p:[1.08239243],tr:[1.3551482],pr:[1.64455128],rev:[2.72351575],dbuf:[10.2538998],ch:[14.69241599],ibuf:[46.0]
962:r:[-13.07467113],p:[1.30746711],tr:[1.61031031],pr:[1.7653204],rev:[3.11582549],dbuf:[13.07608718],ch:[14.66879157],ibuf:[25.0]
963:r:[-11.43252886],p:[1.14325289],tr:[1.2733218],pr:[1.76088757],rev:[2.74869646],dbuf:[6.66905801],ch:[13.33650545],ibuf:[22.5]
964:r:[-12.22478754],p:[1.22247875],tr:[1.77350271],pr:[1.58873206],rev:[2.9357902],dbuf:[7.77490326],ch:[13.71553881],ibuf:[20.0]
965:r:[-12.11707842],p:[1.21170784],tr:[1.6924174],pr:[1.68044082],rev:[2.97509053],dbuf:[7.69439696],ch:[14.04033469],ibuf:[41.5]
966:r:[-13.57979902],p:[1.3579799],tr:[1.52731043],pr:[1.79493218],rev:[2.80486919],dbuf:[8.82819306],ch:[12.37226112],ibuf:[32.0]
967:r:[-14.19115012],p:[1.41911501],tr:[1.60406465],pr:[1.76153905],rev:[3.06513182],dbuf:[10.58458108],ch:[11.29325135],ibuf:[37.5]
968:r:[-13.57728806],p:[1.35772881],tr:[1.37340289],pr:[1.79008957],rev:[3.05424